## Google Earth Engine Code for obtaining the Mean Urban Heat Index Value for London Wards
This document is part of the coursework titled (Urban Heat Island Dynamics in London Wards - The Role of Built Environment Factors) for the QM Module (CASA0007) at UCL CASA.  Github Repository [here](https://github.com/burhanucl2023/QM_Assignment)

### Purpose of this Code

This document focuses on generating our dependent variable for our study i.e. mean Urban Heat Index.
The following Google Earth Engine (GEE) code is used to calculate and export the mean urban heat index values for each ward during summer period of 2005.

**1. Data Used in this Code:**
   - **Landsat Image Collection Dataset**: We used the Landsat 5 Dataset available on Google Earth Engine to obtain the Temperature Values corresponding to the Thermal Band (B6) and eventually caluclate mean UHI per ward.
   - **Ward Data**: After data cleaning and wrangling [here](https://nbviewer.org/github/burhanucl2023/QM_Assignment/blob/main/Urban_Heat_Island_London_QM_Final.ipynb) , we obtain the Spatial Ward Data which has Built Environment Values for each ward and we use the same here for obtaining the mean_uhi values for each ward.

**2. Objectives of this Code:**
   - Calculate the Mean Urban Heat Index Values for each ward for 2005 (which is the year for which we have the Built Environment Features Data)

**3. Results:**
   - Produce a comprehensive  Mean Urban Heat Index Values dataset, which will be used in the subsequent analysis in this [code](https://nbviewer.org/github/burhanucl2023/QM_Assignment/blob/main/Urban_Heat_Island_London_QM_Final.ipynb) 
   - It generates a csv file with the two attributes i.e. 'GSS_CODE', 'mean_uhi'

This document details the steps and code used, ensuring the analysis is transparent and reproducible.

```javascript
// Use the Shapefile of London Wards.
var table = ee.FeatureCollection("projects/ee-burhanahwani/assets/output_shapefile_london");

// Define coordinates for London
var cor = [-0.1278, 51.5074];
var point = ee.Geometry.Point(cor);

var visParams = {
  min: -0.22,
  max: 0.22,
  palette: [
    'blue',    // Much colder than average
    'cyan',    // Colder than average
    'white',   // Average
    'yellow',  // Warmer than average
    'orange',  // Warmer
    'red',     // Much warmer than average
    'brown'    // Extremely warm
  ]
};

function updateMapAndData(year) {
  Map.clear();
  Map.centerObject(table, 10);
  Map.addLayer(table, {}, 'London Wards');

  var time_start = year + '-05-01';
  var time_end = year + '-09-30';

  // Get Landsat 5 images
  var landsat = ee.ImageCollection("LANDSAT/LT05/C01/T1")
    .select('B6')
    .filterBounds(table)
    .filterDate(time_start, time_end)
    .filter(ee.Filter.calendarRange(5, 9, 'month'))
    .filter(ee.Filter.lt('CLOUD_COVER', 40))
    .map(function(img) {
      // Convert thermal band DN to radiance using correct gain and offset
      var gain = ee.Number(0.05518); // correct gain for Landsat 5
      var offset = ee.Number(1.2378); // correct offset for Landsat 5
      var radiance = img.select('B6').multiply(gain).add(offset);
      
      // Convert radiance to temperature in Kelvin using the correct formula
      var K1 = 607.76;
      var K2 = 1260.56;
      var temperature = radiance.expression(
        '(K2 / log((K1 / radiance) + 1)) - 273.15', {
          'K1': K1,
          'K2': K2,
          'radiance': radiance
        });

      return temperature.rename('temperature')
        .copyProperties(img, img.propertyNames());
    });

  var tir_img = landsat.median();

  Map.addLayer(tir_img.clip(table), {}, 'temperature_median_' + year, false);

  // Calculate mean temperature for the entire ROI
  var tir_mean = tir_img.reduceRegion({
    reducer: ee.Reducer.mean(),
    geometry: table.geometry(),
    scale: 100,
    maxPixels: 1e13
  }).get('temperature');

  // Calculate UHI
  var uhi = tir_img.expression('(temperature - mean) / mean', {
    'temperature': tir_img,
    'mean': ee.Number(tir_mean)
  }).rename('uhi');

  Map.addLayer(uhi.clip(table), visParams, 'uhi_' + year, false);

  // Calculate mean temperature and UHI for each ward
  var features = table.map(function(ward) {
    var meanTemperature = tir_img.reduceRegion({
      reducer: ee.Reducer.mean(),
      geometry: ward.geometry(),
      scale: 100,
      maxPixels: 1e13
    }).get('temperature');

    var meanUHI = uhi.reduceRegion({
      reducer: ee.Reducer.mean(),
      geometry: ward.geometry(),
      scale: 100,
      maxPixels: 1e13
    }).get('uhi');

    return ward.set({
      year: year,
      mean_temperature: meanTemperature,
      mean_uhi: meanUHI
    });
  });

  // Store the features globally for exporting
  exportFeatures = features;

  // Print the features for debugging
  features.evaluate(function(f) {
    print('Features:', f);
  });
  // Export the UHI map as a high-quality PNG image with a white background
  var uhiVisual = uhi.visualize(visParams).clip(table);

  var background = ee.Image(1).mask(0).clip(table).visualize({
    palette: ['white'],
    forceRgbOutput: true
  });

  var finalImage = ee.ImageCollection([background, uhiVisual]).mosaic();

  Export.image.toDrive({
    image: finalImage,
    description: 'UHI_Map_' + year,
    scale: 30, // Use a smaller scale for higher resolution
    region: table.geometry().bounds(),
    fileFormat: 'GEOTIFF'
  });
}


var exportFeatures;

function exportData(year) {
  if (!exportFeatures) {
    print('No data to export. Please update the map first.');
    return;
  }

  Export.table.toDrive({
    collection: exportFeatures,
    description: 'London_UHI_Data_By_Ward_' + year,
    fileFormat: 'CSV',
    selectors: ['GSS_CODE', 'mean_uhi']
  });
}

// Initialize the map with the year 2005
updateMapAndData(2005);
exportData(2005);
